In [56]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, boxscoresummaryv2
import time
import pandas as pd
import numpy as np

In [77]:
nba_teams = teams.get_teams()
team_ids = [team['id'] for team in nba_teams]

In [46]:
team_dfs = []
for id in team_ids:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable=id, season_nullable='2019-20').get_data_frames()[0]
    reg_games = games[games.GAME_DATE >= '2019-10-22'].copy()
    team_dfs.append(reg_games)
    time.sleep(5)
all_2019_reg_games = pd.concat(team_dfs)

In [79]:
%%time
qtr1_3_dfs = []
for id in all_2019_reg_games.GAME_ID.unique():
    sum_data = boxscoresummaryv2.BoxScoreSummaryV2(id)
    qtr_data = sum_data.get_data_frames()[5][['GAME_ID', 'TEAM_ID', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3']]
    qtr1_3_dfs.append(qtr_data)
    time.sleep(1)
qtr1_3_df = pd.concat(qtr1_3_dfs)

Wall time: 35min 19s


In [85]:
all_2019_reg_games = all_2019_reg_games.merge(qtr1_3_df, on=['GAME_ID', 'TEAM_ID']).copy()

In [140]:
spreads = pd.read_csv('C:/Users/philb/Google Drive/Thinkful/Thinkful_repo/projects/supervised_capstone/Export Data/2019_spreads.csv', index_col=0)

In [141]:
all_2019_reg_games.TEAM_ABBREVIATION.unique()

array(['ATL', 'BOS', 'CLE', 'NOP', 'CHI', 'DAL', 'DEN', 'GSW', 'HOU',
       'LAC', 'LAL', 'MIA', 'MIL', 'MIN', 'BKN', 'NYK', 'ORL', 'IND',
       'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'OKC', 'TOR', 'UTA', 'MEM',
       'WAS', 'DET', 'CHA'], dtype=object)

In [142]:
spreads.Team.unique()

array(['MIL', 'MEM', 'OKC', 'UTA', 'LAC', 'MIN', 'HOU', 'CHI', 'ATL',
       'PHI', 'NY', 'DET', 'CLE', 'IND', 'ORL', 'BOS', 'DAL', 'DEN',
       'PHO', 'SAC', 'GS', 'SAN', 'CHR', 'TOR', 'BKN', 'MIA', 'LAL',
       'WAS', 'POR', 'NOP'], dtype=object)

In [143]:
spreads['Team'] = spreads.Team.str.replace('GS', 'GSW').replace('NY', 'NYK').replace('PHO', 'PHX').replace('SAN', 'SAS').replace('CHR','CHA')

In [144]:
spreads['Date'] = spreads.Date.astype('datetime64')

In [145]:
spreads.columns = ['GAME_DATE', 'TEAM_ABBREVIATION', 'Spread']

In [146]:
all_2019_reg_games['GAME_DATE'] = all_2019_reg_games.GAME_DATE.astype('datetime64')

In [149]:
all_data = all_2019_reg_games.merge(spreads, on= ['GAME_DATE', 'TEAM_ABBREVIATION']).copy()

In [151]:
all_data.to_csv('C:/Users/philb/Google Drive/Thinkful/Thinkful_repo/projects/supervised_capstone/Export Data/2019-2020_data.csv')